In [ ]:
%pip install -U diffusers transformers accelerate sentencepiece protobuf nltk openai
%pip install flash_attn --no-build-isolation

In [ ]:
import torch
from diffusers import FluxPipeline

flux_pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16) 

flux_pipe.enable_model_cpu_offload() #save some VRAM by offloading the model to CPU. Remove this if you have enough GPU power

In [7]:
import random

def gen_img(prompt, num_inference_steps=4, height=1024, width=2048):
    return flux_pipe(
        prompt,
        max_sequence_length=512,
        num_inference_steps=num_inference_steps,
        generator=torch.Generator("cpu").manual_seed(random.randint(0, 999999999))
    ).images[0]

In [8]:
from openai import OpenAI

openai_api_key = "<OPENAI-API-KEY>"

client = OpenAI(api_key=openai_api_key)

def gen_text(prompt):
    completion = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
      ]
    )
    
    return completion.choices[0].message.content

In [9]:
import asyncio
import re
import json
from datetime import datetime
import os
import random

def extract_json(text):
    pattern = r'({[\s\S]*})'

    search_res = re.search(pattern, text)
    if search_res:
        result = search_res.group(1)
        answer_json = json.loads(result)
        return answer_json
    else:
        raise Exception("No JSON found in the string")

In [40]:
def gen_texture(planets_data):
    print("Generating textures")

    # random_words = gen_random_words_string()
    the_prompt = f"""Generate short one paragraph description of realistic 2d texture of the exoplanet surface based on provided physical data of planets. The image will be generated by Text To Image AI model based on the description that you provide, it must be a 2d texture to cover a 3d sphere to create a 3d representation of the exoplanet so describe it in the description. 
    Do not include planet name in the description. Also describe that it is a 2d texture of a planet surface. Make sure that there is correlation between appereances texture_description, photo_description, settlement_description
    
    The exoplanet data is:
    {planets_data}

    Return descriptions for Text To Image AI model in the format:
    {{
        "descriptions": [
            {{"planet_name": "<name of the exoplanet>", "texture_description": "<description for generating exoplanet texture 2d map>", "photo_description": "<description of the photo from the exoplanet>", "settlement_description": "<description of photo of possible human settlement on the exoplanet taking into physical data of the exoplanet>"}}
        ]
    }}
    """

    text_answer = gen_text(prompt=the_prompt)

    print(f"Got answer:\n\n{text_answer}\n\n")
    print("Extracting json")

    json_answer = extract_json(text_answer)

    print("Generating images")
    for planet_description in json_answer["descriptions"]:
        print(planet_description)
        planet_name2 = planet_description['planet_name'].replace(" ", "_")
        texture_image = gen_img(planet_description["texture_description"], height=1024, width=2048)
        img_path = f"textures/texture_{planet_name2}.png"
        texture_image.save(img_path)

        photo_image = gen_img(planet_description["photo_description"], height=1024, width=1024)
        img_path = f"textures/photo_{planet_name2}.png"
        photo_image.save(img_path)
        
        settlement_image = gen_img(planet_description["settlement_description"], height=1024, width=2048)
        img_path = f"textures/settlement_{planet_name2}.png"
        settlement_image.save(img_path)
        
        print("Saving json")
        json_answer["prompt"] = the_prompt
        json_answer["planet_data"] = planets_data[planet_description['planet_name']]

        with open(f"jsons/json_{planet_name2}.txt", "w") as file:
            file.write(json.dumps(json_answer))

In [ ]:
exoplanets_data_example = {
    "HIP 73786 b": {
        "description": "HIP 73786 b is a confirmed exoplanet discovered in 2010. It orbits a K-type star at a large distance of 1260 AU. The planet was detected using the imaging method and has a calculated temperature of 914 K. It is also known by the alternate names GJ 576 b and ULAS J150457.65+053800.8.",
        "mass": None,
        "radius": None,
        "orbital_period": None,
        "semi_major_axis": "1260.0",
        "eccentricity": None,
        "temperature": 914.0,
        "gravity": None,
        "density": None,
        "habitability": False,
        "surface_conditions": "Unknown, likely gaseous due to its large orbit and temperature",
        "age": 1.6,
        "distance_from_earth": 19.0,
        "travel_time": 285000.0,
        "discovered_method": "Imaging",
        "exoplanet_type": "Gas Giant",
        "star_name": "HIP 73786",
        "star_type": "K-type",
        "star_mass": "0.64",
        "star_radius": "0.596",
        "star_temperature": 4062.0,
        "star_age": 1.6,
        "planet_texture": None,
        "star_texture": None,
        "surface_photos": None,
        "locals_portrait": None,
        "flora_photos": None,
        "camp_photo": None,
        "background": None,
        "publications": [
            {
                "link": "https://doi.org/10.1051/0004-6361/201014264",
                "description": "Hip 63510C, Hip 73786B, and nine new isolated high proper motion T dwarf candidates from UKIDSS DR6 and SDSS DR7"
            },
            {
                "link": "https://doi.org/10.3847/1538-4365/aaf6af",
                "description": "Preliminary Trigonometric Parallaxes of 184 Late-T and Y Dwarfs and an Analysis of the Field Substellar Mass Function into the 'Planetary' Mass Regime"
            }
        ]
    }
}


gen_texture(exoplanets_data_example)